In [46]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

import math
import time

In [47]:
reviewsDF = pd.read_csv('beer_reviews.csv')

In [48]:
reviews_per_beerid = reviewsDF.groupby('beer_beerid').review_overall.count().to_frame('Reviews_count').sort_values(by = "Reviews_count", ascending=False)
filtered_reviews_per_beerid = reviews_per_beerid[reviews_per_beerid['Reviews_count'] > 10]

filteredReviewsDF_by_beer = reviewsDF[reviewsDF['beer_beerid'].isin(filtered_reviews_per_beerid.index)]

reviews_per_user = reviewsDF.groupby('review_profilename').review_overall.count().to_frame('Reviews_count').sort_values(by = "Reviews_count", ascending=False)
filtered_reviews_per_user = reviews_per_user[reviews_per_user['Reviews_count'] > 10]

filteredReviewsDF = filteredReviewsDF_by_beer[filteredReviewsDF_by_beer['review_profilename'].isin(filtered_reviews_per_user.index)]

filteredReviewsDF = filteredReviewsDF.dropna(subset=['review_overall', 'review_aroma', 'review_appearance', 'review_profilename', 'review_palate', 'review_taste', 'beer_name', 'beer_beerid'])

filteredReviewsDF['review_overall'] = filteredReviewsDF['review_overall'].apply(lambda x: math.floor(x))
filteredReviewsDF.dropna(inplace=True)

filteredReviewsDF.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
10,163,Amstel Brouwerij B. V.,1010963392,3,2.0,3.0,fodeeoz,Light Lager,2.5,2.5,Amstel Light,3.5,436
18,163,Amstel Brouwerij B. V.,1010861086,2,3.0,3.0,jdhilt,Light Lager,2.0,2.0,Amstel Light,3.5,436
19,1075,Caldera Brewing Company,1325478004,4,4.5,3.0,UCLABrewN84,Rauchbier,4.0,4.5,Rauch Ür Bock,7.4,58046
20,1075,Caldera Brewing Company,1325360812,4,4.0,4.0,zaphodchak,Rauchbier,3.0,4.0,Rauch Ür Bock,7.4,58046
21,1075,Caldera Brewing Company,1322506304,4,4.5,4.0,Tilley4,Rauchbier,3.5,4.0,Rauch Ür Bock,7.4,58046


In [49]:
profileNames = filteredReviewsDF['review_profilename'].unique().tolist()
boostedReviewsDF = pd.DataFrame(index=filteredReviewsDF.index, columns=filteredReviewsDF.columns)
boostedReviewsDF.dropna(inplace=True)
boostedReviewsDF['Boosted'] = boostedReviewsDF['review_overall']

boostedReviewsDF.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,Boosted


In [ ]:
for profile in profileNames:
    profileDF = filteredReviewsDF[filteredReviewsDF['review_profilename'] == profile]
    profileDF.sort_values(by='review_time', ascending=True)
    index = profileDF.index

    profileDF['Boosted'] = profileDF['review_overall']

    previousIndex = index[0]

    for i in index:
        if (previousIndex == i):
            continue

        profileDF.loc[i, 'Boosted'] = profileDF.loc[previousIndex, 'Boosted'] + profileDF.loc[i, 'Boosted']
        previousIndex = i
    
    
    boostedReviewsDF = boostedReviewsDF.append(profileDF)

<ipython-input-50-f0fcc1ca27d2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profileDF['Boosted'] = profileDF['review_overall']
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [57]:
boostedReviewsDF.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,Boosted
10,163,Amstel Brouwerij B. V.,1010963392,3,2.0,3.0,fodeeoz,Light Lager,2.5,2.5,Amstel Light,3.5,436,3
53684,81,Heineken Nederland B.V.,1013799042,3,2.0,3.0,fodeeoz,Euro Pale Lager,2.5,2.5,Heineken Lager Beer,5.0,246,6
91325,140,Sierra Nevada Brewing Co.,1010961818,4,5.0,4.5,fodeeoz,American Pale Ale (APA),4.5,5.0,Sierra Nevada Pale Ale,5.6,276,10
96343,140,Sierra Nevada Brewing Co.,1013106080,3,3.5,3.0,fodeeoz,American Pale Wheat Ale,3.5,3.5,Sierra Nevada Wheat Beer,4.4,282,13
123281,273,SweetWater Brewing Company,1077762315,4,3.0,3.5,fodeeoz,American Pale Ale (APA),4.5,4.5,Sweetwater 420 Extra Pale Ale,5.4,751,17


In [56]:
boostedReviewsDF.to_csv("boosted_beer_reviews.csv")